In [1]:
from modules.crawlers import subtitles

crawler = subtitles.Subtitles()

In [2]:
shigatsu_subs = crawler.get_subtitle_df('shigatsu wa kimi no uso')
shigatsu_subs

,anime_name,filename,start_time,end_time,content
0,shigatsu wa kimi no uso,shigatsu wa kimi no uso 001.srt,0 days 00:01:13.493000,0 days 00:01:15.493000,（かをり）ん？ うん？
1,shigatsu wa kimi no uso,shigatsu wa kimi no uso 001.srt,0 days 00:01:17.497000,0 days 00:01:19.166000,（鳴き声）
2,shigatsu wa kimi no uso,shigatsu wa kimi no uso 001.srt,0 days 00:01:19.166000,0 days 00:01:21.401000,あっ… ウフフフ。
3,shigatsu wa kimi no uso,shigatsu wa kimi no uso 001.srt,0 days 00:01:21.401000,0 days 00:01:24.404000,フフフフ…。
4,shigatsu wa kimi no uso,shigatsu wa kimi no uso 001.srt,0 days 00:01:24.404000,0 days 00:01:26.406000,おっ。
...,...,...,...,...,...
8193,shigatsu wa kimi no uso,[commie] shigatsu wa kimi no uso - 15 [9976b92...,0 days 00:20:59.859000,0 days 00:21:03.863000,私の脚でしょ！\n立って！ 立ってよ！
8194,shigatsu wa kimi no uso,[commie] shigatsu wa kimi no uso - 15 [9976b92...,0 days 00:21:03.863000,0 days 00:21:05.863000,立て！
8195,shigatsu wa kimi no uso,[commie] shigatsu wa kimi no uso - 15 [9976b92...,0 days 00:21:07.867000,0 days 00:21:10.867000,ううっ うっ…。
8196,shigatsu wa kimi no uso,[commie] shigatsu wa kimi no uso - 15 [9976b92...,0 days 00:21:13.873000,0 days 00:21:17.873000,ああー！！


In [3]:
shigatsu_subs.to_csv('data/subtitles.csv', sep=';', index=None)